In [1]:
import numpy as np
import kagglehub
import os
from collections import defaultdict


In [2]:
if not os.path.exists("/content/data"):
  !kaggle datasets download -d hojjatk/mnist-dataset -p /content
  !unzip /content/mnist-dataset.zip -d /content/data
  !rm /content/mnist-dataset.zip

In [43]:
import numpy as np
import struct

def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

# Assuming the file is in the /content/mnist-dataset directory
images = read_idx('/content/data/train-images-idx3-ubyte/train-images-idx3-ubyte')
labels = read_idx('/content/data/train-labels-idx1-ubyte/train-labels-idx1-ubyte')

print(images.shape)  # Print the shape of the image data
print(labels.shape)  # Print the shape of the label datan

(60000, 28, 28)
(60000,)


In [31]:
idxInImage = [np.where(labels == i)[0] for i in range(10)]
idxInImage

[array([    1,    21,    34, ..., 59952, 59972, 59987]),
 array([    3,     6,     8, ..., 59979, 59984, 59994]),
 array([    5,    16,    25, ..., 59983, 59985, 59991]),
 array([    7,    10,    12, ..., 59978, 59980, 59996]),
 array([    2,     9,    20, ..., 59943, 59951, 59975]),
 array([    0,    11,    35, ..., 59968, 59993, 59997]),
 array([   13,    18,    32, ..., 59982, 59986, 59998]),
 array([   15,    29,    38, ..., 59963, 59977, 59988]),
 array([   17,    31,    41, ..., 59989, 59995, 59999]),
 array([    4,    19,    22, ..., 59973, 59990, 59992])]

In [33]:
class Neuron:
    def __init__(self, pixelNum):
        self.pixelNum = pixelNum
        self.connections = defaultdict(int)

In [34]:
neurons = [Neuron(i) for i in range(28*28)]

In [61]:
# Set the pixel threshold
pixelThreshold = 100

# Get image dimensions (MNIST images are 28x28)
image_height, image_width = images.shape[1], images.shape[2]

# Find all pixel indices where the value > pixelThreshold
image_idx, row_idx, col_idx = np.where(images > pixelThreshold)

# Convert (row, col) to a single pixel index
pixel_nums = row_idx * image_width + col_idx

# Get unique image indices and where they appear
unique_images, start_indices = np.unique(image_idx, return_index=True)

# Split pixel numbers efficiently for each image
image_pixel_lists = np.split(pixel_nums, start_indices[1:])

# Create a dictionary mapping image index -> list of pixel numbers
image_pixel_dict = dict(zip(unique_images, image_pixel_lists))

# Print pixel numbers for the first few images
for img_id in sorted(image_pixel_dict.keys())[:5]:  # Print for first 5 images
    print(f"Image {img_id}: {image_pixel_dict[img_id]}")

Image 0: [156 157 158 160 161 162 163 179 180 181 182 183 184 185 186 187 188 189
 190 204 205 206 207 208 209 210 211 212 213 232 233 234 235 236 237 238
 239 240 241 261 262 263 264 265 269 291 292 319 320 321 348 349 377 378
 379 380 406 407 408 409 435 436 437 438 465 466 467 493 494 495 519 520
 521 522 523 545 546 547 548 549 550 551 571 572 573 574 575 576 577 598
 599 600 601 602 603 623 624 625 626 627 628 629 649 650 651 652 653 654
 655 656 676 677 678 679 680 681 682]
Image 1: [128 129 130 155 156 157 158 159 182 183 184 185 186 187 209 210 211 212
 213 215 216 217 235 236 237 238 239 240 241 243 244 245 263 264 265 266
 267 268 269 272 273 290 291 292 293 296 300 301 317 318 319 320 328 329
 330 344 345 346 356 357 358 372 373 384 385 386 399 400 401 412 413 414
 427 428 429 440 441 442 455 456 467 468 469 483 484 494 495 496 511 512
 521 522 523 539 540 547 548 549 550 567 568 569 573 574 575 576 577 595
 596 597 598 599 600 601 602 603 604 623 624 625 626 627 628 629 630